In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

df_train=pd.read_csv("train.csv",header=0)
df_train=df_train.iloc[:,:80]
df_train.loc[:,"flag"]=0
df_test=pd.read_csv("test.csv",header=0)
df_test.loc[:,"flag"]=1
df=pd.concat([df_train,df_test])
df.index=df.loc[:,'Id']

## Creating dummy variables

In [40]:
col_nos_dummies=('MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
                 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 
                 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
                 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
                 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
                 'Functio0l', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive',
                 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition')
col_nos_continuous=(df.columns-col_nos_dummies)
df_categorical=pd.DataFrame(df.iloc[:,1:2])
for i in range(len(col_nos_dummies)):
    temp_df=pd.get_dummies(df.loc[:,col_nos_dummies[i]])
    df_categorical=pd.concat([df_categorical,temp_df.iloc[:,1:]],axis=1)
df_categorical=df_categorical.iloc[:,2:]
df=pd.concat([df.loc[:,col_nos_continuous],df_categorical],axis=1)
df=df.drop(['Id'],axis=1)

C:\Users\harin\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()


## Random forest classifier

In [56]:
rf=RandomForestClassifier(min_samples_split=4,max_depth=6,min_samples_leaf=10)
X_train=pd.DataFrame.as_matrix(df[df.flag==0])
df_train=pd.read_csv("train.csv",header=0)
y_train=pd.DataFrame.as_matrix(df_train.loc[:,'SalePrice'])
rf.fit(X_train,y_train)

y_pred=rf.predict(X_train)
df_1=pd.DataFrame(y_pred,y_train)
df_1=df_1.reset_index()
df_1.columns=["Predicted","Actual"]
np.mean(abs(df_1.Predicted-df_1.Actual)/df_1.Actual)

X_test=pd.DataFrame.as_matrix(df[df.flag==1])
y_pred=rf.predict(X_test)
y_pred=pd.Series(y_pred)
y_pred.to_csv("result.csv")

## Simple linear regression 

In [61]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression()
regr.fit(X_train,y_train)
y_pred=regr.predict(X_test)
y_pred=pd.Series(y_pred)
y_pred.to_csv("result.csv")

## LASSO regression

In [85]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=20,max_iter=10000)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
y_pred=pd.Series(y_pred)
y_pred.to_csv("result.csv")

C:\Users\harin\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [92]:
coef_lasso=pd.Series(clf.coef_,df.columns)

In [94]:
coef_lasso.to_csv("Analysis.csv")